# Проект «Определение стоимости автомобилей»

### Описание задачи

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение, чтобы привлечь новых клиентов.
В нём можно будет узнать рыночную стоимость своего автомобиля.

Нам необходимо построить модель, которая умеет её определять.

### Описание данных

В нашем распоряжении имеются данные о технических характеристиках, комплектации и ценах других автомобилей.

##### Признаки

* DateCrawled — дата скачивания анкеты из базы
* VehicleType — тип автомобильного кузова
* RegistrationYear — год регистрации автомобиля
* Gearbox — тип коробки передач
* Power — мощность (л. с.)
* Model — модель автомобиля
* Kilometer — пробег (км)
* RegistrationMonth — месяц регистрации автомобиля
* FuelType — тип топлива
* Brand — марка автомобиля
* Repaired — была машина в ремонте или нет
* DateCreated — дата создания анкеты
* NumberOfPictures — количество фотографий автомобиля
* PostalCode — почтовый индекс владельца анкеты (пользователя)
* LastSeen — дата последней активности пользователя

##### Целевой признак

* Price — цена (евро)

## Загрузка библиотек

In [1]:
!pip install lightgbm

DEPRECATION: Loading egg at /usr/lib64/python3.11/site-packages/TBB-2021.12.0-py3.11-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try
    zypper install python311-xyz, where xyz is the package
    you are trying to install.
    
    If you wish to install a non-rpm packaged Python package,
    create a virtual environment using python3.11 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-rpm packaged Python application,
    it may be easiest to use `pipx install xyz`, which will manage a
    virtual environment for you. Ins

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

from lightgbm import LGBMRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [3]:
# Примечание: в новой версии библиотеки sklearn вычисление RMSE будет производится иначе (не как указано в теории)
# На всякий случай, добавил оба варианта.

In [4]:
try:
    from sklearn.metrics import mean_squared_error
except:
    from sklearn.metrics import root_mean_squared_error

In [1]:
RANDOM_STATE = 12345

## Загрузка данных

In [6]:
filename = 'autos.csv'

In [7]:
try:
    data = pd.read_csv('/datasets/' + filename)
except:
    data = pd.read_csv(filename)

#### Осмотр данных

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

Видим, что датасет далеко не полный: есть пропуски в нескольких столбцах.
Кроме того, названия столбцов не соответствуют стандартному стилю.

Посмотрим на первые 5 строк датасета.

In [9]:
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Датасет загрузился корректно.
Данные в датасете соответствуют описанию.

#### Переименование столбцов

Приведем названия столбцов к стандартному стилю, в ряде случаев незначительно изменив их названия.

Сначала введем символ «_», поставив его на место предполагаемого пропуска в CamelCase.

In [10]:
data = data.rename(columns={
    'DateCrawled': 'date_crawled',
    'VehicleType': 'vehicle_type',
    'RegistrationYear': 'reg_year',
    'RegistrationMonth': 'reg_month',
    'FuelType': 'fuel_type',
    'DateCreated': 'date_created',
    'NumberOfPictures': 'num_of_pics',
    'PostalCode': 'postal_code',
    'LastSeen': 'last_seen'
})

Затем приведем названия всех столбцов в нижний регистр.

In [11]:
data.columns = map(str.lower, data.columns)

Готово.
Названия столбцов соответствуют стандартному стилю.

## Предобработка данных

Посмотрим еще раз на датасет

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   date_crawled  354369 non-null  object
 1   price         354369 non-null  int64 
 2   vehicle_type  316879 non-null  object
 3   reg_year      354369 non-null  int64 
 4   gearbox       334536 non-null  object
 5   power         354369 non-null  int64 
 6   model         334664 non-null  object
 7   kilometer     354369 non-null  int64 
 8   reg_month     354369 non-null  int64 
 9   fuel_type     321474 non-null  object
 10  brand         354369 non-null  object
 11  repaired      283215 non-null  object
 12  date_created  354369 non-null  object
 13  num_of_pics   354369 non-null  int64 
 14  postal_code   354369 non-null  int64 
 15  last_seen     354369 non-null  object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


Т.к. мы решаем задачу определения цены автомобиля, оставим в датасете признаки, имеющие отношение к состоянию самого автомобиля, а не к пользователю или анкете.
Кроме того, бессмысленнен признак «reg_month», т.к. сам месяц регистрации ни о чем не говорит, т.к. в любом случае должен быть привязан к году.

In [13]:
non_informative_features = ['date_crawled', 'reg_month', 'date_created', 'num_of_pics', 'postal_code', 'last_seen']

In [14]:
data = data.drop(non_informative_features, axis=1)

In [15]:
data.head()

,price,vehicle_type,reg_year,gearbox,power,model,kilometer,fuel_type,brand,repaired
0,480,NaN,1993,manual,0,golf,150000,petrol,volkswagen,NaN
1,18300,coupe,2011,manual,190,NaN,125000,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,gasoline,skoda,no


Посмотрим на уникальные значения признака vehicle_type («тип автомобильного кузова»).

In [16]:
data['vehicle_type'].unique()

array([nan, 'coupe', 'suv', 'small', 'sedan', 'convertible', 'bus',
       'wagon', 'other'], dtype=object)

Скрытых дубликатов нет.

Посмотрим на уникальные значения признака reg_year («год регистрации автомобиля»).

In [17]:
data['reg_year'].sort_values().unique()

array([1000, 1001, 1039, 1111, 1200, 1234, 1253, 1255, 1300, 1400, 1500,
       1600, 1602, 1688, 1800, 1910, 1915, 1919, 1920, 1923, 1925, 1927,
       1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938,
       1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950,
       1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961,
       1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972,
       1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983,
       1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994,
       1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
       2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       2017, 2018, 2019, 2066, 2200, 2222, 2290, 2500, 2800, 2900, 3000,
       3200, 3500, 3700, 3800, 4000, 4100, 4500, 4800, 5000, 5300, 5555,
       5600, 5900, 5911, 6000, 6500, 7000, 7100, 7500, 7800, 8000, 8200,
       8455, 8500, 8888, 9000, 9229, 9450, 9996, 99

Видим, очевидно, аномальные значения года регистрации автомобиля.
Оценим их количество, взяв в качестве нижней границы 1970й год.

In [18]:
print('Доля автомобилей с аномальным годом регистрации: ', 
      round(len(data[(data['reg_year'] < 1970) | (data['reg_year'] > 2019)]) / len(data) * 100, 2), '%')

Доля автомобилей с аномальным годом регистрации:  0.41 %


Процент очень мал, поэтому просто удалим их.

In [19]:
data = data[(data['reg_year'] > 1970) & (data['reg_year'] <= 2019)]

Посмотрим на уникальные значения признака gearbox («тип коробки передач»)

In [20]:
data['gearbox'].unique()

array(['manual', 'auto', nan], dtype=object)

Скрытых повторов нет.

Посмотрим на признак power («мощность»)

In [21]:
data['power'].describe()

count    352593.000000
mean        110.279733
std         187.916640
min           0.000000
25%          69.000000
50%         105.000000
75%         143.000000
max       20000.000000
Name: power, dtype: float64

Выберем в качестве верхней границы значение в 500 л.с., что соответствует очень мощному спорткару и оценим число автомобилей в датасете с
двигателем мощностью выше этого значения.

In [22]:
print('Доля автомобилей с аномально мощным двигателем: ', 
      round(len(data[data['power'] > 500]) / len(data) * 100, 2), '%')

Доля автомобилей с аномально мощным двигателем:  0.13 %


Удалим эти записи из датасета

In [23]:
data = data[data['power'] < 500]

C автомобилями с двигателями якобы нулевой мощности решим вопрос позднее, на этапе заполнения пропусков.

Модели удобно просматривать применительно к марке автомобиля:

In [24]:
for b in data['brand'].unique():
    print("Марка автомобиля:", b, ":", data[data['brand'] == b]['model'].unique())

Марка автомобиля: volkswagen : ['golf' 'other' 'passat' 'polo' 'scirocco' 'transporter' 'jetta' nan 'eos'
 'touran' 'lupo' 'caddy' 'tiguan' 'sharan' 'up' 'fox' 'beetle' 'touareg'
 'kaefer' 'phaeton' 'cc' 'bora' 'amarok']
Марка автомобиля: audi : [nan 'a8' 'a4' 'a1' 'tt' 'a6' '80' '100' 'a3' 'a2' 'a5' 'other' '90' 'q7'
 'q3' '200' 'q5']
Марка автомобиля: jeep : ['grand' 'wrangler' 'cherokee' 'other' nan]
Марка автомобиля: skoda : ['fabia' 'yeti' 'octavia' 'roomster' 'other' nan 'superb' 'citigo']
Марка автомобиля: bmw : ['3er' '5er' '1er' '7er' 'z_reihe' nan '6er' 'other' 'x_reihe' 'm_reihe'
 'i3']
Марка автомобиля: peugeot : ['2_reihe' '3_reihe' nan 'other' '4_reihe' '1_reihe' '5_reihe']
Марка автомобиля: ford : ['c_max' 'fiesta' 'escort' 'ka' 'focus' 'mustang' 'mondeo' 's_max'
 'galaxy' 'other' 'transit' nan 'fusion' 'kuga' 'b_max']
Марка автомобиля: mazda : ['3_reihe' 'other' '6_reihe' '5_reihe' 'rx_reihe' '1_reihe' 'mx_reihe' nan
 'cx_reihe']
Марка автомобиля: nissan : ['navara' 'mi

Видим, что есть 2 похожие марки автомобиля: rover и land rover.
Хотя автомобили марки rover существуют, у них совсем иной модельный ряд; тут же мы видим, что модельный ряд rover является
подмножеством модельного ряда land rover.
Значит, это на самом деле land rover.
Объединим их, переименовав марку автомобиля rover в land_rover, а также модель rangerover в range_rover (как у land_rover).

In [25]:
data.loc[data['brand'] == 'rover', 'brand'] = 'land_rover'

In [26]:
data.loc[data['model'] == 'rangerover', 'model'] = 'range_rover'

Заметим также, что скрытых повторов в названиях марок автомобилей нет, однако есть марка «sonstige_autos». что в переводе с немецкого
означает «другие автомобили», при этом наименования моделей у этих автомобилей нет.
Восстановить марку не получится, т.е. это фактически неизвестный автомобиль.
Оценим их количество.

In [27]:
print('Доля неизвестных автомобилей: ', 
      round(len(data[data['brand'] == 'sonstige_autos']) / len(data) * 100, 2), '%')

Доля неизвестных автомобилей:  0.83 %


Меньше процента — удалим их.

In [28]:
data = data[data['brand'] != 'sonstige_autos']

Оценим пробег.

In [29]:
data['kilometer'].describe()

count    349196.000000
mean     128799.628289
std       37204.572925
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: kilometer, dtype: float64

Аномальных значений нет.

Посмотрим на тип используемого топлива

In [30]:
data['fuel_type'].unique()

array(['petrol', 'gasoline', nan, 'lpg', 'other', 'hybrid', 'cng',
       'electric'], dtype=object)

Видим скрытый повтор: gasoline и petrol суть разные названия бензина.
Переименуем gasoline в petrol.

In [31]:
data.loc[data['fuel_type'] == 'gasoline', 'fuel_type'] = 'petrol'

Посмотрим признак, отражающий факт нахождения машины в ремонте

In [32]:
data['repaired'].unique()

array([nan, 'yes', 'no'], dtype=object)

Повторов нет.

##### Заполнение пропусков

Заполним пропуски.

Посмотрим на описательную статистику признака price («цена»)

In [33]:
data['price'].describe()

count    349196.000000
mean       4403.699710
std        4493.706561
min           0.000000
25%        1099.000000
50%        2700.000000
75%        6350.000000
max       20000.000000
Name: price, dtype: float64

Мы видим аномалию в виде автомобиля нулевой стоимости, что эквивалентно бесплатному автомобилю.

Возьмем в качестве нижней границы цену в 50 евро — такие объявления можно встретить и установим ее для всех автомобилей стоимостью
ниже 50 евро.

In [34]:
data.loc[data['price'] < 50, 'price'] = 50

У нас в датасете есть 3 взаимосвязанных признака: марка автомобиля, модель и тип кузова.
Отметим при этом, что в признаке «марка автомобиля» пропусков нет.

Сначала оценим количество автомобилей в датасете с отсутствующими моделью и типом кузова, т.к. по одной марке мы не восстановим эти
показатели.

In [35]:
print('Доля автомобилей с отсутствующими моделью и типом кузова: ', 
      round(len(data[data['vehicle_type'].isna() & data['model'].isna()]) / len(data) * 100, 2), '%')

Доля автомобилей с отсутствующими моделью и типом кузова:  1.62 %


Менее 2 процентов. Удалим их из датасета.

In [36]:
data = data[data['vehicle_type'].notna() | data['model'].notna()]

Заполним пропуски в vehicle_type и model следующим образом:
* Пропуск в vehicle_type заполним самым популярным типом кузова для данного производителя и модели
* Пропуск в model заполним самым популярной моделью для данного производителя и типа кузова

In [37]:
vehicle_dict = {}
no_vehicle_list = []
def func(row):
    try:
        row['vehicle_type'] = vehicle_dict[row['brand'] + row['model']]
    except:
        try:
            row['vehicle_type'] = data[(data['brand'] == row['brand']) & (data['model'] == row['model'])]['vehicle_type'].value_counts().index[0]
            vehicle_dict[row['brand'] + row['model']] = row['vehicle_type']
        except:
            no_vehicle_list.append(row['brand'] + ' ' + row['model'])
    
    return row

t_data_veh = data[data['vehicle_type'].isna()].apply(func, axis=1)

In [38]:
model_dict = {}
no_model_list = []
def func(row):
    try:
        row['model'] = model_dict[row['brand'] + row['vehicle_type']]
    except:
        try:
            row['model'] = data[(data['brand'] == row['brand']) & (data['vehicle_type'] == row['vehicle_type'])]['model'].value_counts().index[0]
            model_dict[row['brand'] + row['vehicle_type']] = row['model']
        except:
            no_model_list.append(row['brand'] + ' ' + row['vehicle_type'])
    
    return row

t_data_mod = data[data['model'].isna()].apply(func, axis=1)

Проверим, есть ли типы кузова или модели, для которых отсутствует статистика в заполненной части датасета:

In [39]:
no_vehicle_list

[]

In [40]:
no_model_list

['lada bus', 'smart bus']

Посмотрим эти записи

In [41]:
data[(data['brand'] == 'lada') & (data['vehicle_type'] == 'bus')]

,price,vehicle_type,reg_year,gearbox,power,model,kilometer,fuel_type,brand,repaired
18286,3500,bus,2007,NaN,0,NaN,150000,NaN,lada,NaN


In [42]:
data[(data['brand'] == 'smart') & (data['vehicle_type'] == 'bus')]

,price,vehicle_type,reg_year,gearbox,power,model,kilometer,fuel_type,brand,repaired
293969,1650,bus,2000,NaN,0,NaN,150000,NaN,smart,NaN


Всего две записи.
Удалим их.

In [43]:
data.drop(index=[18286, 293969], inplace=True, axis=1)

In [44]:
t_data_mod.drop(index=[18286, 293969], inplace=True, axis=1)

Запишем заполненные данные в датасет:

In [45]:
data.loc[data['vehicle_type'].isna()] = t_data_veh

In [46]:
data.loc[data['model'].isna()] = t_data_mod

Пропуски в признаке gearbox («тип коробки передач») заполним значением unknown.

In [47]:
data['gearbox'] = data['gearbox'].fillna('unknown')

Нулевую мощность двигателя (чего не может быть) заменим на медианную мощность двигателя для данной марки и модели.

In [48]:
mean_power_dict = {}
no_mean_power_list = []
def func(row):
    try:
        row['power'] = mean_power_dict[row['brand'] + row['vehicle_type']]
    except:
        try:
            row['power'] = round(data[(data['brand'] == row['brand']) &
                                (data['vehicle_type'] == row['vehicle_type']) &
                                (data['power'] != 0)]['power'].median())
            mean_power_dict[row['brand'] + row['vehicle_type']] = row['power']
        except:
            no_mean_power_list.append(row['brand'] + ' ' + row['vehicle_type'])
        
    return row

data.loc[data['power'] == 0] = data[data['power'] == 0].apply(func, axis=1)

Проверим, есть ли автомобили, для мощности двигателя которых отсутствует статистика в заполненной части датасета:

In [49]:
no_mean_power_list

['porsche small', 'land_rover bus', 'kia convertible', 'saab small']

In [50]:
data[(data['brand'] == 'porsche') & (data['vehicle_type'] == 'small')]

,price,vehicle_type,reg_year,gearbox,power,model,kilometer,fuel_type,brand,repaired
1053,140,small,1986,unknown,0,cayenne,20000,petrol,porsche,NaN


In [51]:
data[(data['brand'] == 'land_rover') & (data['vehicle_type'] == 'bus')]

,price,vehicle_type,reg_year,gearbox,power,model,kilometer,fuel_type,brand,repaired
215212,450,bus,1997,auto,0,other,150000,lpg,land_rover,no


In [52]:
data[(data['brand'] == 'kia') & (data['vehicle_type'] == 'convertible')]

,price,vehicle_type,reg_year,gearbox,power,model,kilometer,fuel_type,brand,repaired
249839,10000,convertible,1999,unknown,0,other,20000,petrol,kia,NaN


In [53]:
data[(data['brand'] == 'saab') & (data['vehicle_type'] == 'small')]

,price,vehicle_type,reg_year,gearbox,power,model,kilometer,fuel_type,brand,repaired
261288,2350,small,1971,unknown,0,other,5000,petrol,saab,NaN


Это всё единичные записи, удалим их.

In [54]:
data.drop(index=[1053, 215212, 249839, 261288], inplace=True, axis=1)

Пропуски в fuel_type («тип топлива») и repaired («») заполним значением unknown

In [55]:
data['fuel_type'] = data['fuel_type'].fillna('unknown')

In [56]:
data['repaired'] = data['repaired'].fillna('unknown')

##### Удаление дубликатов

Посмотрим, есть ли полные дубликаты в датасете.

In [57]:
data.duplicated().sum()

np.int64(47473)

Дубликатов много.
Удалим их

In [58]:
data = data.drop_duplicates()

Датасет готов. Предобработка данных завершена.

## Подготовка выборок 

In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 296053 entries, 0 to 354368
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   price         296053 non-null  int64 
 1   vehicle_type  296053 non-null  object
 2   reg_year      296053 non-null  int64 
 3   gearbox       296053 non-null  object
 4   power         296053 non-null  int64 
 5   model         296053 non-null  object
 6   kilometer     296053 non-null  int64 
 7   fuel_type     296053 non-null  object
 8   brand         296053 non-null  object
 9   repaired      296053 non-null  object
dtypes: int64(4), object(6)
memory usage: 24.8+ MB


In [60]:
feature_cat_cols_ohe = ['vehicle_type', 'gearbox', 'fuel_type', 'brand', 'repaired']

In [61]:
feature_cat_cols_oe = ['model']

In [62]:
feature_num_cols = ['reg_year', 'power', 'kilometer']

In [63]:
X = data.drop('price', axis=1)

In [64]:
y = data['price']

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

In [66]:
# В новой версии библиотеки sklearn изменено наименование параметра (не как в теории).
# На всякий случай, добавил оба варианта

In [67]:
try:
    ohe_encoder = OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False) 
except:
    ohe_encoder = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)

In [68]:
X_train_ohe = ohe_encoder.fit_transform(X_train[feature_cat_cols_ohe])
X_test_ohe = ohe_encoder.transform(X_test[feature_cat_cols_ohe])

In [69]:
# В новой версии библиотеки sklearn изменено наименование параметра (не как в теории).
# На всякий случай, добавил оба варианта

In [70]:
try:
    X_train_ohe = pd.DataFrame(X_train_ohe, columns=ohe_encoder.get_feature_names_out())
    X_test_ohe = pd.DataFrame(X_test_ohe, columns=ohe_encoder.get_feature_names_out())
except:
    X_train_ohe = pd.DataFrame(X_train_ohe, columns=ohe_encoder.get_feature_names())
    X_test_ohe = pd.DataFrame(X_test_ohe, columns=ohe_encoder.get_feature_names())

Кодируем модель автомобиля методом OrdinalEncoder.

In [71]:
oe_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

In [72]:
X_train_oe = oe_encoder.fit_transform(X_train[feature_cat_cols_oe])
X_test_oe = oe_encoder.transform(X_test[feature_cat_cols_oe])

In [73]:
X_train_oe = pd.DataFrame(X_train_oe, columns=oe_encoder.get_feature_names_out())
X_test_oe = pd.DataFrame(X_test_oe, columns=oe_encoder.get_feature_names_out())

Масштабируем числовые признаки.

In [74]:
scaler = StandardScaler()

In [75]:
X_train_scaled = scaler.fit_transform(X_train[feature_num_cols])
X_test_scaled = scaler.transform(X_test[feature_num_cols])

In [76]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_num_cols)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_num_cols)

Объединим в окончательные выборки:

In [77]:
X_train = pd.concat([X_train_ohe, X_train_oe, X_train_scaled], axis=1)
X_test = pd.concat([X_test_ohe, X_test_oe, X_test_scaled], axis=1)

Выборки готовы.

## Обучение моделей

### LightGBM

Рассмотрим модель LightGBM и найдем лучшие гиперпараметры для нее.

In [78]:
lgbm = LGBMRegressor()

Зададим параметры

In [79]:
params = {
    'n_estimators': [50, 100, 200],
    'num_leaves': [10, 50, 70, 100],
    'random_state': [RANDOM_STATE]
}

In [80]:
grid = GridSearchCV(
    lgbm,
    params,
    scoring = 'neg_root_mean_squared_error'
)

In [81]:
%%time
grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.best_score_ * (-1))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005157 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 661
[LightGBM] [Info] Number of data points in the train set: 177631, number of used features: 58
[LightGBM] [Info] Start training from score 4549.114102
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007194 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 662
[LightGBM] [Info] Number of data points in the train set: 177631, number of used features: 58
[LightGBM] [Info] Start training from score 4535.933745
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005128 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not

Видим хорошее значение RMSE на кросс-валидации (ниже целевого порога в 2500)

Видим, что лучшие значения параметров следующие: n_estimators — 200, num_leaves — 100.

### Решающее дерево

Обучим модель решающего дерева и получим лучшие гиперпараметры для него.

In [82]:
model_dtreer = DecisionTreeRegressor()

In [83]:
params = {
    'max_depth': [2, 5, 10],
    'random_state': [RANDOM_STATE]
}

In [84]:
%%time
grid = GridSearchCV(
    model_dtreer,
    params,
    scoring = 'neg_root_mean_squared_error'
)

CPU times: user 0 ns, sys: 440 μs, total: 440 μs
Wall time: 249 μs


In [85]:
grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.best_score_ * (-1))

{'max_depth': 10, 'random_state': 12345}
2135.017032626328


Видим в целом удовлетворительный показатель RMSE на кросс-валидации (ниже 2500), но это хуже, чем у LightGBM.

### Случайный лес

Обучим модель случайного леса и найдем лучшие значения гиперпараметров для нее.

In [86]:
model_rforr = RandomForestRegressor()

In [87]:
params = {
    'n_estimators': [10],
    'max_depth': [2, 5, 10],
    'random_state': [RANDOM_STATE]
}

In [88]:
grid = GridSearchCV(
    model_rforr,
    params,
    scoring = 'neg_root_mean_squared_error'
)

In [89]:
%%time
grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.best_score_ * (-1))

{'max_depth': 10, 'n_estimators': 10, 'random_state': 12345}
2043.6355362742474
CPU times: user 36.1 s, sys: 52.5 ms, total: 36.2 s
Wall time: 36.2 s


Видим, что значение RMSE на кросс-валидации удовлетворительно (ниже 2500) и лучше, чем у решающего дерева, но хуже, чем у LightGBM.

### Оценка работы моделей

Посмотрим как ведут себя модели с наилучшими параметрами

#### LightGBM

##### Время обучения

In [90]:
%%time
lgbm = LGBMRegressor(n_estimators=200, num_leaves=100, random_state=RANDOM_STATE)
lgbm.fit(X_train, y_train);

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011211 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 663
[LightGBM] [Info] Number of data points in the train set: 222039, number of used features: 58
[LightGBM] [Info] Start training from score 4541.964006
CPU times: user 5.25 s, sys: 3.08 s, total: 8.33 s
Wall time: 1.71 s


Обучение проходит достаточно быстро.

##### Время предсказания

In [91]:
%%time
lgbm.predict(X_test)

CPU times: user 1.27 s, sys: 0 ns, total: 1.27 s
Wall time: 139 ms


array([ 3174.61746432,   716.39590857, 11940.21313838, ...,
        6234.29462152,  2934.9294939 ,  2199.35846479])

#### Решающее дерево

##### Время обучения

In [92]:
%%time
model_dtreer = DecisionTreeRegressor(max_depth=10, random_state=RANDOM_STATE)
model_dtreer.fit(X_train, y_train);

CPU times: user 843 ms, sys: 4.76 ms, total: 847 ms
Wall time: 844 ms


Время обучения дольше, чем у LightGBM.

##### Время предсказания

In [93]:
%%time
model_dtreer.predict(X_test)

CPU times: user 17.2 ms, sys: 0 ns, total: 17.2 ms
Wall time: 15.8 ms


array([ 2896.7367573 ,   847.85572687, 13007.91428571, ...,
        5836.0466067 ,  2981.26468507,  2745.92647059])

Предсказания у решающего дерева очень быстрое.

#### Случайный лес

##### Время обучения

In [94]:
%%time
model_rforr = RandomForestRegressor(max_depth=10, n_estimators=10, random_state=RANDOM_STATE)
model_rforr.fit(X_train, y_train);

CPU times: user 4.78 s, sys: 0 ns, total: 4.78 s
Wall time: 4.78 s


Время обучения заметно дольше всех.

##### Время предсказания

In [95]:
%%time
model_rforr.predict(X_test)

CPU times: user 68.7 ms, sys: 0 ns, total: 68.7 ms
Wall time: 63.2 ms


array([ 3411.30352002,   882.8218209 , 11442.91708599, ...,
        6010.95953014,  2987.04081463,  2772.02866479])

Предсказание проходит довольно быстро.

С точки зрения качества, то на кросс-валидации заметно лучший RMSE продемонстрировала модель LightGBM.
Выберем ее в качестве лучшей и оценим ее качество на тестовой выборке.

#### Качество лучшей модели на тестовой выборке

Оценим RMSE у LightGBM на тестовой выборке.

In [96]:
try:
    rmse = mean_squared_error(y_test, lgbm.predict(X_test), squared=False)
except:
    rmse = root_mean_squared_error(y_test, lgbm.predict(X_test))

rmse

/usr/lib64/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


np.float64(1728.4586438802528)

Качество модели удовлетворяет условию.

## Вывод

Был проведен анализ данных и получены следующие результаты:

На этапе осмотра и предобработки данных в представленном датасете были исправлены названия столбцов (приведены к стандартному стилю),
обнаружены и исправлены скрытые повторы данных, были заполнены пропуски в данных, а также удалены явные дубликаты.

Были обучены 3 модели регрессии — LightGBM, решающее дерево и случайный лес — и найдены лучшие значения гиперпараметров этих моделей.
Лучшее значение RMSE продемонстрировала модель LightGBM.

С точки зрения скорости работы, то модель LightGBM продемонстрировала приемлемое время обучения и предсказания, тогда как у случайного леса
время обучения заметно дольше всех.

В качестве лучшей была выбрана модель LightGBM, т.к. имеет оптимальные характеристики времени обучения и предсказания, а также лучший RMSE.
Проверка качества на тестовой выборке также продемонстрировала хороший результат с точки зрения качества по метрике RMSE.